In [127]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
import pandas as pd
import data_pecarn 
import data_psrc
import data
import util
from style import *
from collections import Counter
from data import feats_numerical, feats_categorical, meta, outcome_def

# plt.style.use('dark_background')
# sns.set(style="white")
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# benchmark bivariable associations

In [131]:
df_pecarn, df_psrc, common_feats, filtered_feats_pecarn, filtered_feats_psrc = data.load_it_all(dummy=False, impute=False)
d = df_pecarn # [common_feats]
d['Age<2'] = d['Age'] < 2
data.select_final_feats(common_feats)

(array([ 1.,  2.,  3., ..., nan, nan, nan]), array([1635, 1543,  419, ...,    1,    1,    1]))
(array(['Mild', 'Moderate', 'None', 'Severe', 'unknown'], dtype=object), array([1635, 1543,   45,  419, 8402]))


/accounts/projects/vision/.local/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


['AbdDistention',
 'AbdTenderDegree',
 'AbdTrauma',
 'AbdTrauma_or_SeatBeltSign',
 'AbdomenPain',
 'Age',
 'CostalTender',
 'DecrBreathSound',
 'DistractingPain',
 'GCSScore_Full',
 'Hypotension',
 'InitHeartRate',
 'InitSysBPRange',
 'MOI',
 'SeatBeltSign',
 'ThoracicTrauma',
 'VomitWretch']

In [126]:
# can also check vars that were not used
rules = [
    ('AbdTrauma_or_SeatBeltSign', ['yes']),
    ('GCSScore', range(14)),
    ('AbdTenderDegree', ['Mild', 'Moderate', 'Severe']),
    ('ThoracicTrauma', ['yes']),        
    ('AbdomenPain', ['yes']),
    ('DecrBreathSound', ['yes']),
    ('VomitWretch', ['yes']),
    
    # not used in rule
    ('Age<2', [True]),    
    ('AbdDistention', ['yes']),
    ('MOI', ['Motor vehicle collision']),
    ('DistractingPain', ['yes']),    
    ('CostalTender', ['yes']),
    ('ThoracicTender', ['yes']),    
    ('Hypotension', ['yes']),        
]
index = ['NA', 'IAIw/I', 'No IAIw/I', 'Difference']
o = data.outcome_def
r = {k: [] for k in [r[0] for r in rules]}
io = d[o] == 1
# print(d[io])
print('num IwI and key / num IwI (not nan) \t num without IwI and key / num without IwI (not nane) ')
for rule in rules:
    k, vals = rule
    ik = d[k].isin(vals) # d[o]==1
    inan = d[k].isna() | (d[k] == 'unknown')
    r[k].append(np.sum(inan))   
    num1 = np.sum(d[ik][o]==1)
    denom1 = (io & ~inan).sum()
    r[k].append(f'{num1}/{denom1}')
    num2 = np.sum(d[ik][o]==0)
    denom2 = (~io & ~inan).sum()
    r[k].append(f'{num2}/{denom2}')
    r[k].append(f'{num1/denom1 - num2/denom2:0.3f}')    
r = pd.DataFrame.from_dict(r)
r.index = index
r.transpose()

num IwI and key / num IwI (not nan) 	 num without IwI and key / num without IwI (not nane) 


,NA,IAIw/I,No IAIw/I,Difference
AbdTrauma_or_SeatBeltSign,0,112/203,1851/11841,0.395
GCSScore,118,65/200,902/11726,0.248
AbdTenderDegree,8402,113/114,3484/3528,0.004
ThoracicTrauma,76,66/201,1976/11767,0.160
AbdomenPain,148,105/201,3515/11695,0.222
DecrBreathSound,233,25/197,231/11614,0.107
VomitWretch,472,49/165,1104/11407,0.200
Age<2,0,10/203,1157/11841,-0.048
AbdDistention,528,49/199,228/11317,0.226
MOI,476,91/194,3741/11374,0.140


# recreate / evaluate rule

In [91]:
def pecarn_rule_predict(d, o=data.outcome_def):
    n = d.shape[0]
    npos = d[o].sum()
    print(f'{"Initial":<25} {npos} / {n}')
    risks = np.array([np.nan] * d.shape[0])
    rules = [
        ('AbdTrauma_or_SeatBeltSign', ['yes']),
        ('GCSScore', range(14)),
        ('AbdTenderDegree', ['Mild', 'Moderate', 'Severe']),
        ('ThoracicTrauma', ['yes']),        
        ('AbdomenPain', ['yes']),
        ('DecrBreathSound', ['yes']),
        ('VomitWretch', ['yes']),
    ]
    for rule in rules:
        k, vals = rule
        idxs = d[k].isin(vals)
        do = d[idxs]
        d = d[~idxs]
        num2 = do[o].sum()
        denom2 = do.shape[0]
        print(f'{k:<25} {d[o].sum()} / {d.shape[0]}\t{num2} / {denom2} ({num2/denom2*100:0.1f})')
    
    low_risk_patients = d
    patients_missed = low_risk_patients[low_risk_patients[o] == 1]
    
    # calc metrics
    fn = patients_missed.shape[0]
    tp = npos - fn
    tn = low_risk_patients.shape[0] - fn
    fp = (n - low_risk_patients.shape[0]) - tp
    stats = {
        'fn': fn,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'sensitivity': tp / npos * 100,
        'specificity': tn / (tn + fp) * 100.
    }
    return d, patients_missed, stats

low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_pecarn)
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')
missed_patients[['Age', 'Sex', 'MOI']]

Initial                   203 / 12044
AbdTrauma_or_SeatBeltSign 91 / 10081	112 / 1963 (5.7)
GCSScore                  55 / 9279	36 / 802 (4.5)
AbdTenderDegree           19 / 6767	36 / 2512 (1.4)
ThoracicTrauma            13 / 5807	6 / 960 (0.6)
AbdomenPain               11 / 5478	2 / 329 (0.6)
DecrBreathSound           9 / 5443	2 / 35 (5.7)
VomitWretch               7 / 5045	2 / 398 (0.5)
sensitivity 96.55
specificity 42.55


,Age,Sex,MOI
84,4,F,Motor vehicle collision
203,2,M,Fall down stairs
4349,2,M,Pedestrian/bicyclist struck by moving vehicle
5977,16,M,Motorcycle/ATV/Scooter collision
9419,17,F,Motor vehicle collision
9436,17,M,Motor vehicle collision
9971,17,M,Motor vehicle collision


In [69]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_psrc)
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')
missed_patients[['Age', 'MOI']]

Initial                   62 / 2188
AbdTrauma_or_SeatBeltSign 18 / 1708	44 / 480 (9.2)
GCSScore                  8 / 1441	10 / 267 (3.7)
AbdTenderDegree           2 / 1216	6 / 225 (2.7)
ThoracicTrauma            2 / 1123	0 / 93 (0.0)
AbdomenPain               2 / 993	0 / 130 (0.0)
DecrBreathSound           2 / 993	0 / 0 (nan)
VomitWretch               2 / 937	0 / 56 (0.0)
sensitivity 96.77
specificity 43.98


/accounts/projects/vision/.local/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars


,Age,MOI
1164,1.0,Motor vehicle collision
1755,6.0,Fall from an elevation
